In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
from statsmodels.tsa.vector_ar.vecm import coint_johansen

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
data = sm.datasets.webuse('lutkepohl2', 'https://www.stata-press.com/data/r12/')
data.index = data.qtr
data.index.freq = data.index.inferred_freq

In [6]:
data.head()

,inv,inc,consump,qtr,ln_inv,dln_inv,ln_inc,dln_inc,ln_consump,dln_consump
qtr,,,,,,,,,,
1960-01-01,180,451,415,1960-01-01,5.192957,NaN,6.111467,NaN,6.028278,NaN
1960-04-01,179,465,421,1960-04-01,5.187386,-0.005571,6.142037,0.030570,6.042633,0.014355
1960-07-01,185,485,434,1960-07-01,5.220356,0.032970,6.184149,0.042111,6.073044,0.030411
1960-10-01,192,493,448,1960-10-01,5.257495,0.037139,6.200509,0.016360,6.104793,0.031749
1961-01-01,211,509,459,1961-01-01,5.351858,0.094363,6.232448,0.031939,6.129050,0.024257


In [7]:
data.shape

(92, 10)

In [8]:
data = data.dropna()

In [9]:
data.shape

(91, 10)

In [11]:
##determinant order---> det_order -->-1,0,1
#k_ar_diff = log diff
joh_model = coint_johansen(data[['ln_inv','ln_inc','ln_consump']], -1, 1)

In [12]:
joh_model

In [17]:
#trace
#eigen value
joh_model.lr1

array([58.22535787, 13.31987083,  5.2392883 ])

In [14]:
joh_model.lr2

array([44.90548704,  8.08058253,  5.2392883 ])

In [31]:
def joh_output(res):
  output=pd.DataFrame([res.lr2, res.lr1], index=['max_eign','trace_stats'])
  print(output.T,'\n')
  print('Critical Values (90%, 95%, 99%) of max_eign\n', res.cvm,'\n')
  print('Critical Values (90%, 95%, 99%) of max_eign\n', res.cvt,'\n')

In [32]:
joh_output(joh_model)

    max_eign  trace_stats
0  44.905487    58.225358
1   8.080583    13.319871
2   5.239288     5.239288 

Critical Values (90%, 95%, 99%) of max_eign
 [[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]] 

Critical Values (90%, 95%, 99%) of max_eign
 [[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]] 



In [33]:
#58.22>29.51 ===> We can reject null hypothesesis ===> r=0t
#13>16.363 ====> We cannot reject ===> r=1

In [34]:
trace = joh_model.lr1
maxeig = joh_model.lr2
cvt = joh_model.cvt
cvm = joh_model.cvm

In [37]:
r=0
for i in range(3):
  if trace[i]>cvt[i,1]:
    r=r+1

In [38]:
r

3

In [40]:
from statsmodels.tsa.vector_ar.vecm import select_coint_rank

In [41]:
rank = select_coint_rank(data[['ln_inv','ln_inc','ln_consump']],-1,1)

In [42]:
rank.rank

3

In [43]:
from statsmodels.tsa.stattools import coint

In [45]:
coint(data['ln_inv'],data['ln_inc']) ## Null Hypothesis: There is no cointegration

(-3.0484553808716353,
 0.09913190328554355,
 array([-4.02226691, -3.40486235, -3.09191025]))

In [46]:
# If p<0.05 reject the null hypothesis. Here we failed to reject